In [ ]:
%matplotlib inline
import glob

from functools import partial
from multiprocessing import Pool

import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import numpy as np
import typhon.plots
from netCDF4 import Dataset
from typhon.utils import image2mpeg


plt.style.use(typhon.plots.styles('typhon'))

vmin = 180
vmax = 320

# fascod_seasons = [
#     'tropical',
#     'midlatitude-summer',
#     'midlatitude-winter',
#     'subarctic-summer',
#     'subarctic-winter',
# ]

fascod_seasons = ['tropical-moistconvective_scale-co2-moistconvective_4']

# Simple timeseries

In [ ]:
for season in fascod_seasons:
    data = Dataset('results/{}.nc'.format(season))

    fig, ax = plt.subplots()
    sm = ax.pcolormesh(
        data.variables['time'][:] / 24,
        data.variables['plev'][:],
        data.variables['T'][:].T,
        cmap=plt.get_cmap('temperature', 20),
        rasterized=True,
        vmin=vmin,
        vmax=vmax,
    )
    fig.colorbar(sm, label='Temperature [K]')
    ax.invert_yaxis()
    ax.set_yscale('log')
    ax.set_ylim(1100e2, 0.1e2)
    ax.yaxis.set_major_formatter(typhon.plots.HectoPascalLogFormatter())
    ax.set_title('FASCOD {}'.format(season))
    ax.set_xlabel('Days')
    ax.set_ylabel('Pressure [hPa]')
    ax.set_xlim(right=500)
    #fig.savefig('plots/{}.pdf'.format(season))

# Timeseries animation

## Temperature profile

In [ ]:
def calculate_toa(data):
    """Calculate radiation budget at top of the atmosphere (TOA).
    
    Parameters:
        data (dict): Dictionary containing short- and longwave fluxes.
        
    Returns:
        ndarray: Radiation budget at TOA.
    """
    lw_flxu = data.variables['lw_flxu'][:, -1]
    lw_flxd = data.variables['lw_flxd'][:, -1]
    sw_flxu = data.variables['sw_flxu'][:, -1]
    sw_flxd = data.variables['sw_flxd'][:, -1]
    return ((sw_flxd + lw_flxd) - (sw_flxu + lw_flxu))


def plot_timestep(time, p, T, toa, timestep=0):
    fig, [ax2, ax1] = plt.subplots(1, 2, sharey=True, figsize=(16, 8),
                                   gridspec_kw={'width_ratios':[1, 3]})

    sm = ax1.pcolormesh(
        time[0:timestep+1],
        p,
        T[:, 0:timestep+1],
        cmap=plt.get_cmap('temperature', 10),
        rasterized=True,
        vmin=vmin,
        vmax=vmax,
    )
    fig.colorbar(sm, label='Temperature [K]', ax=ax1)
    sm = ax1.pcolormesh(
        time[timestep:],
        p,
        T[:, timestep:],
        cmap=plt.get_cmap('Greys_r', 10),
        rasterized=True,
        vmin=vmin,
        vmax=vmax,
    )
    ax1.axvline(time[timestep])  # vertical line to highlight current timestep.
    ax1.invert_yaxis()
    ax1.set_ylim(p.max(), p.min())
    ax1.yaxis.set_major_formatter(typhon.plots.HectoPascalLogFormatter())
    fig.suptitle(f'{season}')
    ax1.set_xlabel('Days')

    # temperature sounding
    typhon.plots.profile_p_log(p, T[:, timestep], ax=ax2)
    ax2.set_ylim(p.max(), p.min())
    ax2.set_xlim(vmin, vmax)
    ax2.set_xlabel('Temperature [K]')
    ax2.grid(True)
    fig.savefig(f'plots/animations/moistconvective/{season}_{timestep:04d}.png', dpi=300)
    plt.close(fig)
    

for season in fascod_seasons:
    data = Dataset('results/{}.nc'.format(season))
    time = data.variables['time'][:] / 24
    p = data.variables['plev'][:]
    T = data.variables['T'][:].T
    toa = calculate_toa(data)
    
    with Pool(20) as pool:
        plot_wrap = partial(plot_timestep, time, p, T, toa)
        pool.map(plot_wrap, range(len(time)))
    del pool
    
image2mpeg(
    glob='plots/animations/moistconvective/tropical-moistconvective_scale-co2-moistconvective_4_*.png',
    outfile='plots/animations/tropical-moistconvective.mp4',
    framerate=60,
)

## Gregory plot

In [ ]:
def calculate_toa(data):
    """Calculate radiation budget at top of the atmosphere (TOA).
    
    Parameters:
        data (dict): Dictionary containing short- and longwave fluxes.
        
    Returns:
        ndarray: Radiation budget at TOA.
    """
    lw_flxu = data.variables['lw_flxu'][:, -1]
    lw_flxd = data.variables['lw_flxd'][:, -1]
    sw_flxu = data.variables['sw_flxu'][:, -1]
    sw_flxd = data.variables['sw_flxd'][:, -1]
    return ((sw_flxd + lw_flxd) - (sw_flxu + lw_flxu))


def plot_gregory(datasets, timestep=0):
    fig, [ax1, ax2] = plt.subplots(ncols=2, figsize=(16, 8),
                                   gridspec_kw={'width_ratios':[1, 3]})

    for name, dataset in datasets.items():
        # unpack variables
        p = dataset['p']
        time = dataset['time']
        T = dataset['T']
        T_sfc = dataset['T_sfc']
        toa = dataset['toa']
            
        # temperature sounding
        typhon.plots.profile_p_log(p, T[:, timestep], linewidth=3, ax=ax1)
    
        # gregory plot
        ax2.plot(T_sfc[:timestep+1] - T_sfc[0], toa[:timestep+1],
                 linestyle='none', marker='o',
                 label=rf'CO$_2 \times$ {name.split("_")[-1]}',
        )
        
    ax1.set_ylim(bottom=p.max(), top=p.min())
    ax1.set_xlim(vmin, vmax)
    ax1.set_xlabel('Temperature [K]')
    ax1.grid(True)
    
    ax2.grid(True)
    ax2.set_xlabel('Surface temperature change [K]')
    ax2.set_xlim(-5, 15)
    ax2.set_ylabel('TOA imbalance [W/m$^2$]')
    ax2.set_ylim(-5, 20)
    ax2.legend(loc='upper left')
    
    fig.savefig(
        f'plots/animations/moistconvective/gregory_{timestep:04d}.png',
        dpi=150,
    )
    plt.close(fig)
    
    
all_seasons = [
    f'tropical-moistconvective_scale-co2-moistconvective_{i}' for i in [0.5, 1, 2, 4, 8]
]

all_datasets = {}
for season in all_seasons:
    tmp_dict = {}
    data = Dataset(f'results/{season}.nc')
    tmp_dict['p'] = data.variables['plev'][:]
    tmp_dict['time'] = data.variables['time'][1:] / 24
    tmp_dict['T'] = data.variables['T'][1:, :].T
    tmp_dict['T_sfc'] = data.variables['temperature'][1:]
    tmp_dict['toa'] = calculate_toa(data)[1:]
    all_datasets[season] = tmp_dict

# # **Commented out so that people do not run this accidentally**
# plot_wrap = partial(plot_gregory, all_datasets)
# with Pool(16) as pool:
#     pool.map(plot_wrap, range(1750))
# del pool
# 
# image2mpeg(
#     glob='plots/animations/moistconvective/gregory_*.png',
#     outfile='plots/animations/tropical-moistconvective_gregory.mp4',
#     framerate=5,
# )